In [1]:
import numpy as np
import pandas as pd
import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

In [4]:
data = pd.read_csv('banner_click_stat.txt', header=None, sep='\t')
data.columns = ['banner_a', 'banner_b']
data.head()

,banner_a,banner_b
0,0,0
1,1,1
2,0,0
3,0,0
4,0,0


In [5]:
data.describe()

,banner_a,banner_b
count,1000.000000,1000.000000
mean,0.037000,0.053000
std,0.188856,0.224146
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,1.000000,1.000000


In [6]:
#   Оценим баннеры через доверительный интервал Уилсона
wilson_a = proportion_confint(sum(data.banner_a), data.shape[0], method='wilson')
wilson_b = proportion_confint(sum(data.banner_b), data.shape[0], method='wilson')
print(f'Banner A stat:\n[{wilson_a[0]:.4f},{wilson_a[1]:.4f}], width={wilson_a[1] - wilson_a[0]:.4f}')

print(f'Banner B stat:\n[{wilson_b[0]:.4f},{wilson_b[1]:.4f}], width={wilson_b[1] - wilson_b[0]:.4f}')

Banner A stat:
[0.0270,0.0506], width=0.0236
Banner B stat:
[0.0407,0.0687], width=0.0279


####   Интервалы пересекаются поэтому возникает проблема с тем как их сравнивать.
####    Для решения такой проблемы можно оценивать разность долей успеха

In [8]:
def subtraction(sample_1, sample_2, alpha=.05):
    p1 = sum(sample_1) / len(sample_1)
    p2 = sum(sample_2) / len(sample_2)
    z_quantile = scipy.stats.norm.ppf(1 - alpha/2)
    add_sub = z_quantile * np.sqrt((p1 * (1 - p1) / len(sample_1)) + (p2 * (1 - p2) / len(sample_2)))
    return p1 - p2 - add_sub, p1 - p2 + add_sub

subtraction(data.banner_a, data.banner_b)

(-0.0341571385110543, 0.002157138511054299)

#### Однозначный ответ дать нельзя. Поэтому предположим что выборки связанные (пр. показываются одним и тем же людям)

In [17]:
def sopr_count(sample_1, sample_2, alpha=.05):
    sample = list(zip(sample_1, sample_2))
    n = len(sample)
    e = sum([1 if x[0] == 1 and x[1] == 1 else 0 for x in sample])
    f = sum([1 if x[0] == 1 and x[1] == 0 else 0 for x in sample])
    g = sum([1 if x[0] == 0 and x[1] == 1 else 0 for x in sample])
    h = sum([1 if x[0] == 0 and x[1] == 0 else 0 for x in sample])
    p1 = e + f / n
    p2 = e + g /n
    z_quantile = scipy.stats.norm.ppf(1 - (alpha / 2))
    add_sub = z_quantile * np.sqrt(((f + g) / (n ** 2)) - ((f - g) ** 2/(n ** 3)))
    return ((f - g) / n) - add_sub, ((f - g) / n) + add_sub

sopr_count(data.banner_a, data.banner_b)

(-0.02668926335921826, -0.00531073664078174)

#   Можно сделать вывод, что новый банер лучше, так как p1 - старый, а p1-p2 лежит левее нуля